# Dog/Cat Classification - Computer Vision Project

I will work with image data related to dogs and cats photos.

The **goal** of this project is to make a model which can recognize a photo as a *Dog* or *Cat*.
For this purpose, a *Convolutional Neural Network* is being used.

## Import the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # i will use it for data augmentation

## Data Preprocessing

The dataset contains 3 directories:
1. **Train Set** : Training images of cat and dogs
2. **Test Set** : Test images of cat and dogs
3. **Single Predictions** : Directory for single image predictions

*Note*: The dataset can be found [here](https://drive.google.com/drive/folders/1qaZ9lrM89hGx_e-Jfev4dyxVkuvLswbZ)

### Preprocess the training set

In [2]:
# Make some transformations ONLY to the training data in order to avoid overfitting (e.g Zoom, rotation,etc.)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64), # equal size for all images
        batch_size=32,
        class_mode='binary') # This code-snippet connects the above object with the image data

Found 7999 images belonging to 2 classes.


### Preprocess the test set

In [3]:
# This time i transform the images by only applying rescaling.

test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


## Build the CNN

In [4]:
model = tf.keras.Sequential([ 
                            tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape = (64,64,3)),
                            tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
                            
                            tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'),
                            tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
                            
                            tf.keras.layers.Flatten(),
                            
                            tf.keras.layers.Dense(128,activation='relu'), #hidden layer
                            tf.keras.layers.Dense(1,activation='sigmoid')])

In [5]:
model.summary(line_length=75)

Model: "sequential"
___________________________________________________________________________
Layer (type)                     Output Shape                  Param #     
conv2d (Conv2D)                  (None, 62, 62, 32)            896         
___________________________________________________________________________
max_pooling2d (MaxPooling2D)     (None, 31, 31, 32)            0           
___________________________________________________________________________
conv2d_1 (Conv2D)                (None, 29, 29, 32)            9248        
___________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 14, 14, 32)            0           
___________________________________________________________________________
flatten (Flatten)                (None, 6272)                  0           
___________________________________________________________________________
dense (Dense)                    (None, 128)                   80294

In [6]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Train the CNN on the training set and evaluating it on the test set

In [7]:
model.fit(training_set,validation_data=test_set,epochs=23,verbose=2)

Epoch 1/23
250/250 - 25s - loss: 0.6842 - accuracy: 0.5631 - val_loss: 0.6407 - val_accuracy: 0.6495
Epoch 2/23
250/250 - 25s - loss: 0.6240 - accuracy: 0.6548 - val_loss: 0.6418 - val_accuracy: 0.6195
Epoch 3/23
250/250 - 25s - loss: 0.5820 - accuracy: 0.6936 - val_loss: 0.5605 - val_accuracy: 0.7130
Epoch 4/23
250/250 - 25s - loss: 0.5453 - accuracy: 0.7168 - val_loss: 0.5359 - val_accuracy: 0.7290
Epoch 5/23
250/250 - 25s - loss: 0.5291 - accuracy: 0.7320 - val_loss: 0.5495 - val_accuracy: 0.7215
Epoch 6/23
250/250 - 25s - loss: 0.5115 - accuracy: 0.7462 - val_loss: 0.5309 - val_accuracy: 0.7440
Epoch 7/23
250/250 - 25s - loss: 0.4962 - accuracy: 0.7605 - val_loss: 0.5085 - val_accuracy: 0.7540
Epoch 8/23
250/250 - 25s - loss: 0.4734 - accuracy: 0.7742 - val_loss: 0.5245 - val_accuracy: 0.7425
Epoch 9/23
250/250 - 25s - loss: 0.4616 - accuracy: 0.7770 - val_loss: 0.4917 - val_accuracy: 0.7625
Epoch 10/23
250/250 - 25s - loss: 0.4468 - accuracy: 0.7870 - val_loss: 0.4785 - val_accura

As we can see, the CNN model has an overall 81% accuracy

## Make single predictions

In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image

#load the image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64,64))

#apply some transformation (that is converting it into array)
test_image = image.img_to_array(test_image)

#convert it into batches (since the model has been trained with batches, rather than with single images)
test_image = np.expand_dims(test_image,axis=0)

result = model.predict(test_image)
print(result)


[[1.]]


But what does it mean 0 and 1?

In [9]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

Thus, 0 means *'Cat'* and 1 *'Dog'*

In [10]:
if result[0][0] == 1:
    prediction = 'Dog'
else:
    prediction = 'Cat'
print(prediction)

Dog


In [11]:
# Create a function to predict the class of an image
def get_prediction(img_path):
    test_image = image.load_img(img_path, target_size=(64,64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    result = model.predict(test_image)
    if result[0][0] == 1:
        return 'Dog'
    else:
        return 'Cat'

In [12]:
get_prediction('dataset/single_prediction/cat_or_dog_2.jpg')

'Cat'

In [13]:
get_prediction('dataset/single_prediction/cat_or_dog_3.jpg')

'Dog'

In [14]:
get_prediction('dataset/single_prediction/cat_or_dog_4.jpg') # False prediction

'Dog'

In [15]:
get_prediction('dataset/single_prediction/cat_or_dog_5.jpg') 

'Cat'